In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True
plt.ion()   # interactive mode

In [5]:
from torchvision import datasets, transforms

# Define the data transformations (you can customize this based on your data)
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to fit AlexNet input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Assuming you have a 'train' folder with subfolders for each class in your dataset
train_dataset = datasets.ImageFolder(root='../Dataset/Cropped', transform=data_transforms)

# Set batch size
batch_size = 32

# Create the data loader
dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')
        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0
        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / 928
                epoch_acc = running_corrects.double() / 928

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [7]:
import torchvision.models as models
alexnet = models.alexnet(pretrained=True)

D:\Programs\Conda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Programs\Conda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
for param in alexnet.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default

num_ftrs = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Linear(num_ftrs, 4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_conv = alexnet.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
optimizer_conv = optim.Adam(model_conv.classifier[6].parameters(), lr=0.0001)

# Learning rate scheduling (optional)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [9]:
torch.cuda.is_available()

False

In [10]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 1.3910 Acc: 0.3254
val Loss: 1.2602 Acc: 0.4321

Epoch 1/24
----------
train Loss: 1.2313 Acc: 0.4709
val Loss: 1.1642 Acc: 0.5183

Epoch 2/24
----------
train Loss: 1.1904 Acc: 0.4591
val Loss: 1.1029 Acc: 0.5377

Epoch 3/24
----------
train Loss: 1.1173 Acc: 0.5420
val Loss: 1.0621 Acc: 0.5787

Epoch 4/24
----------
train Loss: 1.0718 Acc: 0.5636
val Loss: 1.0339 Acc: 0.5873

Epoch 5/24
----------
train Loss: 1.0545 Acc: 0.5722
val Loss: 0.9990 Acc: 0.6433

Epoch 6/24
----------
train Loss: 1.0317 Acc: 0.5991
val Loss: 0.9809 Acc: 0.6239

Epoch 7/24
----------
train Loss: 1.0119 Acc: 0.5884
val Loss: 0.9755 Acc: 0.6293

Epoch 8/24
----------
train Loss: 1.0053 Acc: 0.6056
val Loss: 0.9717 Acc: 0.6369

Epoch 9/24
----------
train Loss: 1.0143 Acc: 0.5894
val Loss: 0.9693 Acc: 0.6422

Epoch 10/24
----------
train Loss: 1.0102 Acc: 0.6024
val Loss: 0.9678 Acc: 0.6390

Epoch 11/24
----------
train Loss: 1.0114 Acc: 0.5787
val Loss: 0.9658 Acc: 0.6466

Ep